In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

# Методы ускорения вычислений

Проведем анализ скорости разных подходов к вычислению

In [ ]:
np.random.seed(12345) 
df = pd.DataFrame({'a': np.random.randn(500_000),
                   'b': np.random.randn(500_000),
                   'N': np.random.randint(100, 1000, (500_000)),
                   'x': 'x'})
def solve(a, b, N):
    return (np.sin(a) + np.cos(b))**2 + N ** (a + b)

Обычный цикл

In [ ]:
%%timeit
answer = []
for row in range(len(df)):
    answer.append(solve(df.loc[row, 'a'], df.loc[row, 'b'], df.loc[row, 'N']))
df["answer"] = answer

19.3 s ± 537 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Цикл по numpy матрице

In [ ]:
%%timeit
answer = []
for row in df.values:
    answer.append(solve(row[0], row[1], row[2]))
df["answer"] = answer

1.8 s ± 33.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


apply

In [ ]:
%%timeit
df["answer"] = df.apply(lambda row: solve(row[0], row[1], row[2]), axis=1)

8.58 s ± 796 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Операция непосредственно над Series

In [ ]:
%%timeit
df["answer"] = solve(df['a'], df['b'], df['N'])

70.2 ms ± 801 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Операция непосредственно над векторами numpy

In [ ]:
%%timeit
df["answer"] = solve(df['a'].values, df['b'].values, df['N'].values)

76.6 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Работаем с данными бейсбольных игр

In [2]:
# отключаем предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')
# cчитываем данные и выводим первые 5 наблюдений
gl = pd.read_csv('https://github.com/aksenov7/Kaggle_competition_group/blob/master/game_logs.zip?raw=true', compression="zip")
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [3]:
gl.dtypes

date                    int64
number_of_game          int64
day_of_week            object
v_name                 object
v_league               object
                       ...   
h_player_9_id          object
h_player_9_name        object
h_player_9_def_pos    float64
additional_info        object
acquisition_info       object
Length: 161, dtype: object

In [4]:
# выводим точную информацию об использовании памяти
gl.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 859.4 MB


# Внутреннее представление датафрейма

In [5]:
gl.shape

(171907, 161)

In [6]:
# посмотрим, сколько памяти в среднем используют 
# столбцы определенного типа
for dtype in ['float','int','object']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Использование памяти в среднем для {} столбцов: {:03.2f} MB".
          format(dtype,mean_usage_mb))

Использование памяти в среднем для float столбцов: 1.29 MB
Использование памяти в среднем для int столбцов: 1.12 MB
Использование памяти в среднем для object столбцов: 9.50 MB


# Подтипы

In [7]:
# посмотрим минимальное и максимальное значения для
# каждого целочисленного типа
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------



# Оптимизация числовых столбцов с помощью понижающего преобразования

In [8]:
# мы будем довольно часто подсчитывать использование памяти,
# поэтому напишем функцию, которая сэкономит нам немного времени

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # предположим, что если это не датафрейм, то серия
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)

# выполняем понижающее преобразование 
# для столбцов типа int
gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(gl_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

7.87 MB
1.48 MB


,before,after
uint8,NaN,5.0
uint32,NaN,1.0
int64,6.0,NaN


In [9]:
# выполняем понижающее преобразование 
# для столбцов типа float
gl_float = gl.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(gl_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([gl_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

100.99 MB
50.49 MB


,before,after
float32,NaN,77.0
float64,77.0,NaN


In [10]:
# создаем копию исходного датафрейма
optimized_gl = gl.copy()

# заменяем исходные числовые столбцы
# оптимизированными
optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float

# смотрим использование памяти
print(mem_usage(gl))
print(mem_usage(optimized_gl))

859.43 MB
802.54 MB


# Сравнение способов хранения числовых и строковых значений

In [11]:
# смотрим размеры отдельно
# хранящихся строк
from sys import getsizeof

s1 = 'working out'
s2 = 'memory usage for'
s3 = 'strings in python is fun!'
s4 = 'strings in python is fun!'

for s in [s1, s2, s3, s4]:
    print(getsizeof(s))

60
65
74
74


In [12]:
# смотрим размеры строк, являющихся
# элементами серии
obj_series = pd.Series(['working out',
                          'memory usage for',
                          'strings in python is fun!',
                          'strings in python is fun!'])
obj_series.apply(getsizeof)

0    60
1    65
2    74
3    74
dtype: int64

In [13]:
# смотрим количество уникальных значений
# по каждому столбцу типа object
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()

,day_of_week,v_name,v_league,h_name,h_league,day_night,completion,forefeit,protest,park_id,...,h_player_6_id,h_player_6_name,h_player_7_id,h_player_7_name,h_player_8_id,h_player_8_name,h_player_9_id,h_player_9_name,additional_info,acquisition_info
count,171907,171907,171907,171907,171907,140150,116,145,180,171907,...,140838,140838,140838,140838,140838,140838,140838,140838,1456,140841
unique,7,148,7,148,7,2,116,3,5,245,...,4774,4720,5253,5197,4760,4710,5193,5142,332,1
top,Sat,CHN,NL,CHN,NL,D,"19200904,,0,6,36",H,V,STL07,...,grimc101,Charlie Grimm,grimc101,Charlie Grimm,lopea102,Al Lopez,spahw101,Warren Spahn,HTBF,Y
freq,28891,8870,88866,9024,88867,82724,1,69,90,7022,...,427,427,491,491,676,676,339,339,1112,140841


In [14]:
gl_obj.day_of_week.nunique()

7

In [15]:
# запишем столбец day_of_week как dow
dow = gl_obj.day_of_week
print(dow.head())

# присваиваем столбцу dow тип category
dow_cat = dow.astype('category')
print(dow_cat.head())

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: object
0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: category
Categories (7, object): ['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed']


In [16]:
# выводим целочисленные значения,
# соответствующие исходным значениям
dow_cat.head().cat.codes

0    4
1    0
2    2
3    1
4    5
dtype: int8

In [17]:
# смотрим, сколько памяти использует столбец dow
# до и после преобразования в тип category
print(mem_usage(dow))
print(mem_usage(dow_cat))

9.84 MB
0.16 MB


In [18]:
converted_obj = pd.DataFrame()

# пишем цикл, которой перебирает каждый столбец object, 
# проверяет его на соответствие заданному порогу 
# (количество уникальных значений должно быть меньше 50% 
# от общего количества значений), и если столбец 
# удовлетворяет порогу, преобразовывает его в тип category
for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]

In [19]:
# снова применяем функцию mem_usage, смотрим,
# сколько памяти занимают все столбцы типа object
# до и после преобразования в тип category
print(mem_usage(gl_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([gl_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

750.57 MB
49.89 MB


,before,after
object,78.0,NaN
category,NaN,2.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
...,...,...
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0


In [20]:
# смотрим, сколько памяти использует датафрейм
# после оптимизации типов
optimized_gl[converted_obj.columns] = converted_obj
mem_usage(optimized_gl)

'101.86 MB'

In [21]:
# смотрим, сколько памяти использует
# столбец date
date = optimized_gl.date
print(mem_usage(date))
date.head()

0.66 MB


0    18710504
1    18710505
2    18710506
3    18710508
4    18710509
Name: date, dtype: uint32

# Задаем типы во время считывания данных

In [22]:
# преобразуем столбец 
# date в тип datetime 
optimized_gl['date'] = pd.to_datetime(date,format='%Y%m%d')

# смотрим, сколько памяти использует
# столбец date
print(mem_usage(optimized_gl))
optimized_gl.date.head()

102.52 MB


0   1871-05-04
1   1871-05-05
2   1871-05-06
3   1871-05-08
4   1871-05-09
Name: date, dtype: datetime64[ns]

In [23]:
# создаем словарь, в котором ключами будут имена
# столбцов, а значениями - типы столбцов
dtypes = optimized_gl.drop('date',axis=1).dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

# вместо того, чтобы печатать 161 столбец,
# выберем 10 пар "ключ-значение" из словаря
# и красиво распечатаем их, используя prettyprint

preview = first2pairs = {key:value for key,value in list(column_types.items())[:10]}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

{   'day_of_week': 'category',
    'h_game_number': 'uint8',
    'h_league': 'category',
    'h_name': 'category',
    'h_score': 'uint8',
    'number_of_game': 'uint8',
    'v_game_number': 'uint8',
    'v_league': 'category',
    'v_name': 'category',
    'v_score': 'uint8'}


In [24]:
# считываем данные с нужными нам типами
path = 'https://github.com/aksenov7/Kaggle_competition_group/blob/master/game_logs.zip?raw=true'
read_and_optimized = pd.read_csv(path,
                                 compression="zip",
                                 dtype=column_types,
                                 parse_dates=['date'],
                                 infer_datetime_format=True)
# смотрим использование памяти
print(mem_usage(read_and_optimized))
read_and_optimized.head()

102.51 MB


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,1871-05-04,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,1871-05-05,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,1871-05-06,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,1871-05-08,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,1871-05-09,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


## Потоковое считывание набора даны

In [25]:
# считываем данные с нужными нам типами
path = 'https://github.com/aksenov7/Kaggle_competition_group/blob/master/game_logs.zip?raw=true'
for chunk in  pd.read_csv(path,
                                 compression="zip",
                                 dtype=column_types,
                                 parse_dates=['date'],
                                 infer_datetime_format=True,
                                 chunksize=100_000):
    # любые преобразования
    print(mem_usage(chunk))
    chunk.to_csv("new_streaming_df.csv", mode="a")
chunk.head()

57.31 MB
46.38 MB


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
100000,1963-09-05,0,Thu,WS2,AL,140,NYA,AL,141,2,...,Johnny Blanchard,7.0,boyec102,Clete Boyer,5.0,terrr101,Ralph Terry,1.0,NaN,Y
100001,1963-09-05,0,Thu,CHN,NL,140,LAN,NL,141,0,...,Ken McMullen,5.0,tracd101,Dick Tracewski,6.0,richp102,Pete Richert,1.0,NaN,Y
100002,1963-09-05,0,Thu,PIT,NL,139,MLN,NL,141,0,...,Denis Menke,5.0,mcmir101,Roy McMillan,6.0,sadob102,Bob Sadowski,1.0,NaN,Y
100003,1963-09-05,0,Thu,HOU,NL,141,SFN,NL,141,5,...,Jim Davenport,5.0,pagaj101,Jose Pagan,6.0,bolib101,Bobby Bolin,1.0,NaN,Y
100004,1963-09-05,0,Thu,NYN,NL,140,SLN,NL,140,0,...,Tim McCarver,2.0,altmg101,George Altman,9.0,simmc101,Curt Simmons,1.0,NaN,Y
